# LSA

In [14]:
import os
import glob
import re
from docx.api import Document
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from natasha import Doc, Segmenter
import numpy as np
import matplotlib.pyplot as plt
import math

## Titles reader

In [15]:
# Путь к файлам с текстами
PATH = os.path.abspath('..\\rfej_parser\\articles\\') + '\\'

In [16]:
# sort file names
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [32]:
def get_titles() -> list:
    """Get all titles from .docx to list.
    """
    list_of_titles = []
    files = [f for f in glob.glob(PATH + '*.docx')]
    # Сортировка названий файлов
    files.sort(key=natural_keys)
    for f in files:
        doc = Document(os.path.abspath(f))
        list_of_titles.append(doc.paragraphs[0].text)
    return list_of_titles

In [18]:
# Получаем список названий всех статей
docs = get_titles()

---

# LSA

In [707]:
from numpy.linalg import svd

In [708]:
class LSA(object):
    """stopwords: words to ignore 
        docs: texts in list
    """
    
    def __init__(self, stopwords, docs):
        self.docs = []
        # Cодержит номера документов, в которых встречается каждое слово
        self.wdict = {}
        # Ключевые слова в матрице
        self.dictionary = []
        # слова которые исключаем из анализа
        self.stopwords = stopwords
        # инициализируем сами документы
        for doc in docs: 
            self.add_doc(doc)

    def prepare(self):
        self.build()
        self.calc()
        
    def tokenizer(self, title: str) -> list:
        # Split sentance into tokens and return list.
        candidate_pos = ['NOUN', 'ADJF', 'VERB', 'INFN', 'PRTS', 'PRTF']
        words_of_sentance = []
        doc = Doc(str(title))
        doc.segment(Segmenter())  # Разбиваем на токены (слова)
        words_raw = [word.text for word in doc.tokens]
        for w in words_raw:
            if morph.parse(w)[0].tag.POS in candidate_pos: # or str(morph.parse(w)[0].tag) == 'LATN':  
                final_word = morph.parse(w)[0].normal_form  # Привести к начальной форме
                # Убрать мусор в 1 символ
                if len(final_word) != 1:
                    words_of_sentance.append(final_word) 
        return words_of_sentance

    def dic(self, word, add = False):
        # если слово есть в словаре возвращаем его номер
        if word in self.dictionary: 
            return self.dictionary.index(word)
        else: 
            # если нет и стоит флаг автоматически добавлять, то пополняем словари возвращвем код слова
            if add:
                self.dictionary.append(word)
                return len(self.dictionary) - 1
            else: 
                return None

    def add_doc(self, doc: str):
        words = [self.dic(word, True) for word in self.tokenizer(doc)]
        if not words:
            return
        self.docs.append(words)
        for word in words:
            if word in self.stopwords:  
                continue
            elif word in self.wdict:
                self.wdict[word].append(len(self.docs) - 1)
            else:                      
                self.wdict[word] = [len(self.docs) - 1]

    def build(self):
        # убираем одиночные слова
        self.keys = [k for k in self.wdict.keys() if len(self.wdict[k]) > 0]
        self.keys.sort()
        # создаём пустую матрицу 
        self.A = np.zeros([len(self.keys), len(self.docs)])
        # наполняем эту матрицу
        for i, k in enumerate(self.keys):
            for d in self.wdict[k]:
                self.A[i,d] += 1

    def calc(self):
        """ Вычисление U, S Vt - матриц """
        self.U, self.S, self.Vt = svd(self.A)

    def TFIDF(self):
        td = self.A
        tf = td / td.sum(axis=0)
        idf = np.log(td.shape[1] / np.count_nonzero(td, axis=1)).reshape(-1, 1)
        self.A = tf * idf

    def dump_src(self):
        self.prepare()
#         print('Здесь представлен расчет матрицы ')
#         for i, row in enumerate(self.A):
#             print (self.dictionary[i], row)

    def print_svd(self):
        self.prepare()
        print('Здесь сингулярные значения')
        print (self.S)
        print('Здесь первые 3 колонки U матрица ')
        for i, row in enumerate(self.U):
            print (self.dictionary[self.keys[i]], row[0:3])
        print ('Здесь первые 3 строчки Vt матрица')
        print (-1*self.Vt[0:3, :])

    def find(self, word):
        word = word.lower()
        self.prepare()
        idx = self.dic(word)
        if not idx:
            print ('Слово не встерчается')
            return []
        if not idx in self.keys:
            print ('Слово отброшено, как не имеющее значения (stopwords)')
            return []
        idx = self.keys.index(idx)
        print ('word:', word)
        # получаем координаты слова
        wx, wy = (-1 * self.U[:, 1:3])[idx]
        print ('coordinates: {}, {:0.2f}, {:0.2f}'.format(idx, wx, wy))
        arts = []
        xx, yy = -1 * self.Vt[1:3, :]
        for k, v in enumerate(self.docs):
            # получаем координаты документа
            ax, ay = xx[k], yy[k]
            #вычисляем расстояние между словом и документом
            dx, dy = float(wx - ax), float(wy - ay)
            arts.append((k, v, ax, ay, math.sqrt(dx * dx + dy * dy)))
            # возвращаем отсортированный по расстоянию список
        return sorted(arts, key = lambda a: a[4])

In [12]:
# Если необходмо добавить стоп-слова: файл `stopwords.txt`
stopwords = set(w.rstrip() for w in open('stopwords.txt', encoding='utf-8'))

In [711]:
# Инициализация класса
lsa = LSA(stopwords, docs)
# Создаем матрицу отсутствия или наличия слова в документе `lsa.A`
lsa.dump_src()
# Нормализация веса слов в матрице
lsa.TFIDF()
# SVD
lsa.calc()
# lsa.print_svd()

### LSA result

In [716]:
# Слово, по-которому мы будем осуществлять поиск в сис.координат
word = "Россия"

In [717]:
# Сравнение документов на оси координат и поиск по ним
result = lsa.find(word)

word: россия
coordinates: 34, -0.00, 0.80


In [720]:
# Результат, отсортированный по возрастанию расстояния от слова к док-ту в сис.координат
# Вывод: номер документа в списке, растояние, документ разложеный на коды, документ
for res in result[:10]:
    print(res[0], res[4], res[1], docs[res[0]])

415 0.7234104344073895 [45, 345, 762, 211, 34, 189, 375, 763, 764] Экономические аспекты военно-технического сотрудничества России со странами Азии и Ближнего Востока*
859 0.7260216733421287 [77, 70, 211, 189, 22, 187, 1183, 34, 4] Экономический потенциал Российской Арктики в области природных ресурсов 
918 0.7268173942477618 [18, 211, 34, 189, 654, 375] Развитие глубокой переработки газа в мировой экономике 
724 0.7285395842621368 [211, 34, 189, 22, 232, 996, 233, 1077] Сотрудничество России и стран БРИКС в области возобновляемой энергетики: биотопливо
65 0.7288315594902037 [94, 211, 52, 114, 34, 139, 226] Цифровое сотрудничество во внешнеэкономической политике России и Республики Корея 
323 0.7291727113066148 [114, 172, 177, 96, 79, 128, 5, 76, 9, 34] Политика Японии в отношении региональных торговых соглашений и влияние на внешнюю торговлю с Россией
1074 0.7299238597978919 [65, 398, 71, 225, 114, 456, 83, 118, 29, 34, 264]  Тенденции и структура взаимной торговли Вьетнама и России
9